# A `CookieCutter` template project

 - Takes care of all the boilerplate for creating a new custom widget project
 - Based on Audrey Roy Greenfeld's (@audreyr) cookiecutter project.
 - URL of the project: https://github.com/jupyter/widget-cookiecutter
 
<img src="cookiecutter.jpg" style="width:200px"></img>

```
pip install cookiecutter
cookiecutter https://github.com/jupyter/widget-cookiecutter.git
```

The cookiecutter project is meant to help custom widget authors get started with the packaging and the distribution of Jupyter interactive widgets.

It produces a project for a Jupyter interactive widget library following the current best practices for using interactive widgets. An implementation for a placeholder "Hello World" widget is provided


# Inline example of custom widget devised with the `%%JavaScript` magic

## The `Hello World` Example of the Cookie Cutter

The widget framework is built on top of the Comm framework (short for communication).  The Comm framework is a framework that allows you send/receive JSON messages to/from the front end (as seen below).

![Widget layer](WidgetArch.png)

To create a custom widget, you need to define the widget both in the browser and on the kernel size.

## Python Kernel

### DOMWidget and Widget

 - `DOMWidget`: Intended to be displayed in the Jupyter notebook
 - `Widget`: A terrible name for a synchronized object. It could not have any visual representation.

### `_view_name`

Inheriting from the DOMWidget does not tell the widget framework what front end widget to associate with your back end widget.  Instead, you must tell it yourself by defining a specially named traitlet, `_view_name` (as seen below).

In [1]:
import ipywidgets as widgets
from traitlets import Unicode


class HelloWidget(widgets.DOMWidget):

    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)

### `sync=True` traitlets

- Traitlets: A  library for defining type-safe properties.

- The `sync=True` tag tells the widget framework to handle synchronizing that value to the browser.
 (Without `sync=True`, the browser would have no knowledge of `_view_name`.)

### Other traitlet types

Unicode, used for _view_name, is not the only Traitlet type, there are many more some of which are listed below:  

- Any
- Bool
- Bytes
- CBool
- CBytes
- CComplex
- CFloat
- CInt
- CLong
- CRegExp
- CUnicode
- CaselessStrEnum
- Complex
- Dict
- DottedObjectName
- Enum
- Float
- FunctionType
- Instance
- InstanceType
- Int
- List
- Long
- Set
- TCPAddress
- Tuple
- Type
- Unicode
- Union


Not all of these traitlets can be synchronized across the network, only the JSON-able traits and Widget instances will be synchronized.

## Front end (JavaScript)

### Models and Views

Jupyter widgets rely on [Backbone.js](http://backbonejs.org/).

Backbone.js is an MVC (model view controller) framework.

Widgets defined in the back end are automatically synchronized with generic Backbone.js models in the front end.  The traitlets are added to the front end instance automatically on first state push.  The `_view_name` trait that you defined earlier is used by the widget framework to create the corresponding Backbone.js view and link that view to the model.

### Import  jupyter-js-widgets, define the view, implement the render method

In [2]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {
    
    var HelloView = widgets.DOMWidgetView.extend({
        
        // Render the view.
        render: function() { 
            this.$el.text('Hello World!'); 
        },
    });
    
    return {
        HelloView: HelloView
    };
});

<IPython.core.display.Javascript object>

## Test

You should be able to display your widget just like any other widget now.

In [3]:
HelloWidget()

A Jupyter Widget

## Making the widget stateful

Instead of displaying a static "hello world" message, we can display a string set by the back end.  

- First you need to add a traitlet in the back end. 

  (Use the name of `value` to stay consistent with the rest of the widget framework and to allow your widget to be used with interact.)

In [4]:
class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)

In [6]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {
    
    var HelloView = widgets.DOMWidgetView.extend({
        
        render: function() { 
            this.$el.text(this.model.get('value')); 
        },
    });
    
    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

### Dynamic updates

 - Adding and registering a change handler.

In [7]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {
    
    var HelloView = widgets.DOMWidgetView.extend({
        
        render: function() { 
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.$el.text(this.model.get('value')); 
        },
    });
    
    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [8]:
w = HelloWidget()
w

A Jupyter Widget

In [9]:
w.value = 'test'

# An example including bidirectional communication: A Spinner Widget

In [10]:
from traitlets import CInt


class SpinnerWidget(widgets.DOMWidget):
    _view_name = Unicode('SpinnerView').tag(sync=True)
    _view_module = Unicode('spinner').tag(sync=True)
    value = CInt().tag(sync=True)

In [11]:
%%javascript
requirejs.undef('spinner');

define('spinner', ["jupyter-js-widgets"], function(widgets) {

    var SpinnerView = widgets.DOMWidgetView.extend({
        render: function() { 

            var that = this;
            this.$input = $('<input />');
            this.$el.append(this.$input);
            this.$spinner = this.$input.spinner({
                change: function( event, ui ) {
                    that.handle_spin(that.$spinner.spinner('value'));
                },
                spin: function( event, ui ) {
                    //ui.value is the new value of the spinner
                    that.handle_spin(ui.value);
                }
            });
            
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.$spinner.spinner('value', this.model.get('value'));
        },
        
        handle_spin: function(value) {
            this.model.set('value', value);
            this.touch();
        },
    });
    
    return {
        SpinnerView: SpinnerView
    };
});

<IPython.core.display.Javascript object>

## Test of the spinner widget

In [12]:
w = SpinnerWidget(value=5)
w

A Jupyter Widget

In [13]:
w.value = 7

## Wiring the spinner with another widget

In [14]:
from IPython.display import display
w1 = SpinnerWidget(value=0)
w2 = widgets.IntSlider()
display(w1,w2)

from traitlets import link
mylink = link((w1, 'value'), (w2, 'value'))

A Jupyter Widget

A Jupyter Widget

# A more advanced example

In [15]:
%%javascript
requirejs.undef('datepicker');

define('datepicker', ["jupyter-js-widgets"], function(widgets) {
    
    // Define the DatePickerView
    var DatePickerView = widgets.DOMWidgetView.extend({
        render: function() {
            this.$el.addClass('widget-hbox-single'); /* Apply this class to the widget container to make
                                                        it fit with the other built in widgets.*/
            // Create a label.
            this.$label = $('<div />')
                .addClass('widget-hlabel')
                .appendTo(this.$el)
                .hide(); // Hide the label by default.
            
            // Create the date picker control.
            this.$date = $('<input />')
                .attr('type', 'date')
                .appendTo(this.$el);
        },
        
        update: function() {
            // Set the value of the date control and then call base.
            this.$date.val(this.model.get('value')); // ISO format "YYYY-MM-DDTHH:mm:ss.sssZ" is required    
            return DatePickerView.__super__.update.apply(this);
        },
        
        // Tell Backbone to listen to the change event of input controls (which the HTML date picker is)
        events: {"change": "handle_date_change"},
        
        // Callback for when the date is changed.
        handle_date_change: function(event) {
            this.model.set('value', this.$date.val());
            this.touch();
        },
    });
    
    return {
        DatePickerView: DatePickerView
    };
});

<IPython.core.display.Javascript object>

In [16]:
from dateutil import parser

class DateWidget(widgets.DOMWidget):
    _view_name = Unicode('DatePickerView').tag(sync=True)
    _view_module = Unicode('datepicker').tag(sync=True)
    
    value = Unicode().tag(sync=True)
    
    # This function automatically gets called by the traitlet machinery when
    # value is modified because of this function's name.
    def _value_changed(self, name, old_value, new_value):
        
        # Parse the date time value.
        try:
            parsed_date = parser.parse(new_value)
            parsed_date_string = parsed_date.strftime("%Y-%m-%d")
        except:
            parsed_date_string = ''
        
        # Set the parsed date string if the current date string is different.
        if self.value != parsed_date_string:
            self.value = parsed_date_string

In [17]:
d = DateWidget(description='Pick a date')
d

A Jupyter Widget

In [18]:
d.value

''